# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [37]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201213_201451_cgan_long_run_model4', '20201211_121…

In [38]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201213_201451_cgan_long_run_model4


## Plot Losses

In [39]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [40]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
93741,93741.0,29.0,0.080008,0.081402,0.161410,4.153800,35.285526,31.131727,-0.983181,31.123243,-1.020283,4.399945,-4.166646,-4.137958,0.143932
93742,93742.0,29.0,0.079869,0.080029,0.159898,4.216934,35.684097,31.467161,-0.779330,30.970627,-1.256595,4.485055,-4.134050,-4.202066,0.145267
93743,93743.0,29.0,0.077796,0.079906,0.157703,4.207718,35.827286,31.619566,-0.792235,31.358131,-1.204562,4.452376,-4.215800,-4.192711,0.151225
93744,93744.0,29.0,0.083695,0.081258,0.164953,4.181289,35.056740,30.875450,-0.780735,32.273369,-0.455576,4.425518,-4.186499,-4.165876,0.150470
93745,93745.0,29.0,0.088504,0.080358,0.168863,4.160446,35.697342,31.536894,-0.602179,32.622650,-0.256618,4.301622,-4.174061,-4.144697,0.148955
93746,93746.0,29.0,0.081495,0.081346,0.162841,4.093177,35.301994,31.208817,-0.724497,31.357815,-0.919041,4.006391,-4.141822,-4.076338,0.147844
93747,93747.0,29.0,0.080015,0.079924,0.159939,4.152900,35.690876,31.537975,-0.960422,31.762802,-0.930175,4.023701,-4.068444,-4.137049,0.142697
93748,93748.0,29.0,0.077737,0.079791,0.157528,4.167509,35.214653,31.047146,-0.573250,31.371235,-1.067424,4.070054,-4.161995,-4.151873,0.142297
93749,93749.0,29.0,0.077396,0.079481,0.156877,4.202797,36.168510,31.965714,-0.979218,31.719030,-1.167090,4.217940,-4.122985,-4.187713,0.149251
93750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.314714,-0.836686,NaN,NaN,NaN,NaN


In [41]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [42]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

0.05942747592926025


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
6944,6944.0,2.0,0.116041,0.360165,0.476206,22.432400,55.522430,33.090031,-3.851370,32.826096,-5.116661,5.599079,-24.616367,-22.432400,0.150024
6947,6947.0,2.0,0.369631,0.239770,0.609401,11.992655,44.346474,32.353821,-3.388175,33.255516,-4.352732,6.342559,-16.771313,-11.992640,0.146932
6959,6959.0,2.0,0.154640,0.533656,0.688297,33.609791,66.101761,32.491966,-3.966184,32.619343,-4.285170,9.309304,-34.498848,-33.609791,0.159041
6943,6943.0,2.0,0.119483,0.436359,0.555841,24.471308,56.515678,32.044369,-4.429832,32.247845,-4.272566,6.010238,-27.029297,-24.471308,0.154210
8298,8298.0,2.0,0.145287,0.263150,0.408436,15.233311,47.596172,32.362862,-3.846033,32.634865,-4.264948,8.949253,-16.655201,-15.233311,0.140718
8299,8299.0,2.0,0.173087,0.210690,0.383776,12.579260,45.712799,33.133537,-3.721652,32.523331,-4.143004,7.224117,-14.284895,-12.579240,0.141285
6948,6948.0,2.0,0.176929,0.216495,0.393424,8.695475,41.102371,32.406898,-3.508928,32.948215,-4.127368,7.563262,-12.320009,-8.695013,0.148088
6958,6958.0,2.0,0.155136,0.543662,0.698798,34.986420,67.581802,32.595387,-3.439243,32.777889,-4.120483,8.418497,-35.269768,-34.986420,0.161299
6301,6301.0,2.0,0.081987,0.082315,0.164302,4.073338,36.356693,32.283356,-3.972258,32.302120,-4.102960,4.007706,-3.974652,-4.055865,0.142963
6962,6962.0,2.0,0.171092,0.369513,0.540605,18.348171,50.544254,32.196083,-3.188931,33.170135,-4.093796,11.840075,-22.342091,-18.348167,0.158446


In [43]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
859,859.0,0.0,0.113463,0.084010,0.197473,4.977925,38.722752,33.744827,-6.530193,33.726425,-7.479975,4.343786,-5.097799,-4.969096,0.141812
860,860.0,0.0,0.084062,0.066368,0.150430,4.599017,38.304630,33.705612,-6.715182,33.715912,-7.445721,4.272352,-4.756011,-4.582138,0.148664
861,861.0,0.0,0.119482,0.093264,0.212746,3.591725,37.283890,33.692165,-6.285973,33.704193,-7.425199,4.244564,-4.506486,-3.560680,0.141602
862,862.0,0.0,0.090234,0.100657,0.190891,3.882569,37.571636,33.689068,-6.248688,33.700314,-7.389379,4.775930,-3.221872,-3.858895,0.141001
864,864.0,0.0,0.075657,0.079591,0.155249,4.733249,38.457409,33.724159,-6.601970,33.716759,-7.375550,4.513812,-4.908442,-4.722061,0.146355
863,863.0,0.0,0.119221,0.079320,0.198541,4.745336,38.419853,33.674519,-6.084863,33.699566,-7.270701,5.359093,-4.148774,-4.735455,0.141662
858,858.0,0.0,0.103859,0.092214,0.196073,4.529841,38.241444,33.711601,-6.586440,33.711922,-6.994156,4.818973,-4.197311,-4.518284,0.139544
865,865.0,0.0,0.075905,0.087028,0.162932,4.214909,37.929581,33.714672,-6.089779,33.695885,-6.944198,5.095245,-4.176728,-4.199518,0.143001


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
92321,92321.0,29.0,0.082950,0.079842,0.162792,4.122169,35.386181,31.264013,-0.412258,30.099115,-0.448687,4.019641,-4.050603,-4.105764,0.149158
70755,70755.0,22.0,0.082767,0.082346,0.165113,4.133841,36.058220,31.924381,1.603871,30.116911,1.473339,4.485311,-4.184681,-4.117659,0.153126
92288,92288.0,29.0,0.082795,0.080683,0.163478,4.209092,34.151585,29.942493,-0.558481,30.117897,-0.600399,4.002345,-4.133827,-4.194070,0.155688
85442,85442.0,27.0,0.078359,0.080794,0.159153,4.183664,35.360909,31.177242,1.325748,30.137424,1.393657,4.150086,-4.161833,-4.168273,0.146940
91451,91451.0,29.0,0.084170,0.081991,0.166162,4.284858,34.555122,30.270264,-0.329082,30.157459,-0.524246,4.309808,-4.577295,-4.270895,0.144611
79326,79326.0,25.0,0.076481,0.081334,0.157815,4.115012,35.098686,30.983675,1.594149,30.160576,1.691631,4.126006,-4.154893,-4.098479,0.143393
91932,91932.0,29.0,0.079721,0.081061,0.160782,4.197451,35.592422,31.394970,-0.395690,30.165968,-0.465087,4.072512,-4.053663,-4.182260,0.148587
92305,92305.0,29.0,0.078629,0.079683,0.158312,4.210530,35.429317,31.218788,-0.487653,30.167728,-0.584203,4.081137,-4.122222,-4.195507,0.146764


## Read stored chi-squares for images

In [17]:

df_list=[]
labels_lst=[0,1,2,3]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(label)))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list

In [18]:

def f_plot_hist_spec(df,class_labels,steps_list,bkg_dict,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    for class_label in class_labels:
        df=df[df.step.isin(steps_list)]
        print(df.shape)
        suffix='_%s'%(class_label)
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(class_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        dict_bkg=bkg_dict[str(class_label)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
    
    if plot_type in ['hist','spec']:     
        plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
        plt.yscale('log')

# f_plot_hist_spec(df_merged,labels_lst,best_step,bkgnd_dict,'hist')

### Read validation data

In [19]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_list=[0.5,0.65,0.8,1.1]
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_list[label])
    samples=np.load(fname,mmap_mode='r+')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(label)]=dict_val
    del samples

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [20]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',labels_lst=[0,1,2,3],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for i in labels_lst: chi_cols.append(j+'_'+str(i))
    else: ## Add chi-square for specific label
        assert label in labels_lst, "label %s is not in %s"%(label,labels_lst)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=-1,use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [21]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),label=ToggleButtons(options=['all']+labels_lst),
labels_lst=fixed(labels_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [22]:
df_sliced=w.result
# df_sliced

In [21]:
best_step=[]
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_spec1',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[21990 22060 22800 28860]


In [24]:
# best_step=[6176]
# best_step=np.arange(40130,40135).astype(int)

In [22]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(4, 79)
[(7, 21990), (7, 22060), (7, 22800), (9, 28860)]


In [23]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
2198,7-21990,183.204103,2024.296841,0.199136
2205,7-22060,171.494012,1962.064800,0.214433
2279,7-22800,125.454715,1829.992997,0.327251
2885,9-28860,145.147480,1170.065880,0.216597


In [24]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                class_labels=SelectMultiple(options=labels_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='class_labels', options=(0, 1, 2, 3), value=()), SelectMultip…

<function __main__.f_plot_hist_spec(df, class_labels, steps_list, bkg_dict, plot_type)>